In [ ]:
%load_ext autoreload
%autoreload 2

import plotly.express as px
import pandas as pd
import numpy as np
from IPython.display import display

from duet.process import *
from duet.constants import *
from common import *

df_prep = preprocess_data(load_raw())
unique_suites = df_prep[RF.suite].unique()

In [ ]:
df = df_prep[
    (
        (df_prep[RF.suite].isin(["dacapo"]))
        & (df_prep[RF.benchmark].isin(["avrora"]))
        & (df_prep[RF.runid] == 0)
        & (df_prep[RF.type].isin(["duet"]))
        & (df_prep[AF.date] == df_prep[AF.date].iloc[0])
        # & (df_prep[RF.pair] == "B")
    )
]
df = df[df[RF.iteration] < df[RF.iteration].min() + 5]

# display(convert_ns(df)[ARTIFACT_COL + ITER_ID_COL + TIME_D_COL])
display(
    px.timeline(
        df,
        x_start=RF.start,
        x_end=RF.end,
        y=RF.pair,
        color=RF.iteration,
        hover_data=[RF.benchmark, RF.pair, RF.runid, RF.time],
        title="Original",
    )
)

slowdown = 0.2
df = alter_score(df, slowdown)
df = convert_ns(df)
# display(df[ARTIFACT_COL + ITER_ID_COL + TIME_D_COL])

px.timeline(
    df,
    x_start=RF.start,
    x_end=RF.end,
    y=RF.pair,
    color=RF.iteration,
    hover_data=[RF.benchmark, RF.pair, RF.runid, RF.time],
    title=f"Slowdown of B by {(slowdown * 100):.0f}%",
)

In [ ]:
slowdown = 0.1
df = alter_score(df_prep, slowdown)
df = compute_ci(df)

for suite in unique_suites:
    suite_mask = df[RF.suite] == suite
    if suite_mask.any():
        fig = px.scatter(
            df[suite_mask],
            x="benchmark",
            y="mid",
            error_y="err",
            color=RF.type,
            facet_col=DF.env,
            title=f"CI with A/B emulated {(slowdown * 100):.0f}% slowdown - {suite}",
        )
        fig.update_xaxes(categoryorder="category ascending")
        fig.show()

In [ ]:
slowdowns = [0.01 * i for i in range(16)]
dfs = []
for slowdown in slowdowns:
    df = alter_score(df_prep, slowdown)
    df_ci = compute_ci(df)
    df_pred = arbiter(df, df_ci)
    df_pred = group_predictions(df_pred)
    df_pred[DF.slowdown] = slowdown
    dfs.append(df_pred)

df_slowdown = pd.concat(dfs)
df_slowdown

In [ ]:
fig = px.line(
    translate(df_slowdown),
    x=DF.slowdown,
    y=DF.match_ratio_ci,
    color=RF.type,
    facet_col=DF.env,
    facet_row=RF.suite,
    markers=True,
    color_discrete_map=colormap,
    category_orders=orders,
)
# Update facet axis description
fig.for_each_xaxis(lambda a: a.title.update(text=""))
fig.for_each_yaxis(lambda a: a.title.update(text=""))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

xaxis_annotation = go.layout.Annotation(
    x=0.5,
    y=-0.08,
    font=dict(size=14),
    showarrow=False,
    text="Slowdown",
    textangle=0,
    xref="paper",
    yref="paper",
)
yaxis_annotation = go.layout.Annotation(
    x=-0.08,
    y=0.5,
    font=dict(size=14),
    showarrow=False,
    text="CI test A/A match",
    textangle=-90,
    xref="paper",
    yref="paper",
)
fig.update_layout(
    legend_title="",
    legend=dict(orientation="h", yanchor="top", xanchor="center", y=1.08, x=0.5),
    annotations=list(fig.layout.annotations) + [xaxis_annotation, yaxis_annotation],
    height=1000,
    width=800,
    template="plotly_white",
)
fig.write_image("figures/mds_citest.pdf")
fig.show()

In [ ]:
fig = px.line(
    translate(df_slowdown[df_slowdown[RF.suite] == "SPEC CPU"]),
    x=DF.slowdown,
    y=DF.match_ratio_utest,
    color=RF.type,
    facet_col=DF.env,
    facet_row=RF.suite,
    markers=True,
    color_discrete_map=colormap,
    category_orders={DF.env: ["bare-metal", "AWS t3.medium"], **order_type},
)
fig = save_fig_facet_col_env(
    fig,
    xaxis_title="Slowdown",
    yaxis_title="u-test A/A match",
    legend_title="",
    filename="figures/mds_utest.pdf",
)
fig.show()